# Custom Regression Models: Implementation and Validation

## Imports

In [ ]:
import linear_regression
import logistic_regression

## Linear Regression: Implementation Check & Evaluation

## Custom Linear Regression vs Scikit-Learn